In [68]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

## Załadowanie danych oczyszczonych i zlematyzowanych oraz dalsze czyszczenie

In [5]:
#sp = pd.read_csv('00_dane/rawest.csv',  sep='|', encoding='UTF-8')
#del sp

In [3]:
#dane = sp[~sp['speech_rawest'].isnull()]

In [4]:
dane = pd.read_csv('00_dane/dane_TM.csv',  sep='|', encoding='UTF-8')

In [12]:
dane.head()

,date_,speech_rawer,full_name,party_final,speech_rawest
0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us..."
1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb..."
2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze..."
3,01-02-1992,"Wszystkim nam bardzo zależy na tym, aby urato...",Alojzy Szablewski,Klub Parlamentarny Zjednoczenia Chrześcijańsko...,"[wszystek, zależeć, uratować, przemysł, stocze..."
4,01-02-1992,Z brzmienia pkt. 1 projektu uchwały odczytuję...,Andrzej Baraniecki,Sojusz Lewicy Demokratycznej,"[brzmię, punkt, projekt, uchwała, odczytować, ..."


In [9]:
%%time
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub('[\W\d]',' ', x.lower()))

#czas dla apply : 19s

Wall time: 33 s


In [10]:
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')

In [11]:
%%time
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])

Wall time: 8.15 s


### Zapisanie pliku w takiej postaci

In [50]:
len(dane)

158879

In [27]:
dane.to_csv('00_dane/dane_TM.csv', sep='|', encoding='UTF-8', index=False)


## Podzbiór testowy (analiza za: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [55]:
#test = dane['speech_rawest'][:50000]

In [44]:
#print(test[113])

['rok', 'trwać', 'praca', 'legislacyjny', 'związany', 'opracowanie', 'nowy', 'ustawa', 'ochrona', 'grunt', 'rolny', 'leśny', 'dzień', 'grudzień', 'rok', 'ustawa', 'zostać', 'uchwalić', 'sejm', 'rzeczypospolity', 'polski', 'zasadniczy', 'sprawa', 'duży', 'osiągnięcie', 'omawiać', 'ustawa', 'decentralizacja', 'decyzja', 'dotyczyć', 'ochrona', 'grunt', 'wykorzystać', 'zamierzyć', 'cel', 'ustawa', 'zostać', 'osiągnąć', 'wyeliminować', 'niekorzystny', 'rolnictwo', 'produkcja', 'rolny', 'uregulowanie', 'prawny', 'dostosować', 'przepis', 'ochrona', 'grunt', 'rolny', 'leśny', 'zmienić', 'uwarunkowanie', 'społeczny', 'gospodarczy', 'ustawa', 'poszerzać', 'swoboda', 'właściciel', 'grunt', 'ograniczać', 'możliwość', 'wywłaszczenie', 'zwiększać', 'samodzielność', 'władza', 'lokalny', 'zmierzać', 'ochrona', 'wartościowy', 'grunt', 'rolny', 'leśny', 'zawierać', 'szereg', 'nowy', 'istotny', 'zdanie', 'klub', 'parlamentarny', 'sojusz', 'lewica', 'demokratyczny', 'rozstrzygnięcie', 'obejmować', 'ustawa

In [17]:
dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])

In [14]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 agent
1 amnestyjny
2 ażeby
3 bezprecensnsowy
4 bezzasadny
5 budżetowy
6 celowo
7 chcieć
8 cierpieć
9 ciągle
10 contry


Usuwanie słów rzadkich i zbyt częstych i ograniczeni liczby tokenów

In [19]:
dictionary.filter_extremes(no_above=0.5, no_below=5 )

In [20]:
len(dictionary)

52425

In [37]:
print(dictionary)

Dictionary(52425 unique tokens: ['agent', 'amnestyjny', 'ażeby', 'bezprecensnsowy', 'bezzasadny']...)


In [ ]:
token2id : dict of (str, int)
    token -> tokenId.
id2token : dict of (int, str)
    Reverse mapping for token2id, initialized in a lazy manner to save memory (not created until needed).
cfs : dict of (int, int)
    Collection frequencies: token_id -> how many instances of this token are contained in the documents.
dfs : dict of (int, int)
    Document frequencies: token_id -> how many documents contain this token.

In [55]:
w = "polski"
print("Słowo {} - {} pojawia się w korpusie {} razy i występuje w {} dokumentach ({:04.1f}%).".format(dictionary.token2id[w],
                                                                                         w,
                                                                                         dictionary.cfs[dictionary.token2id[w]],
                                                                                         dictionary.dfs[dictionary.token2id[w]],
                                                                                         100*dictionary.dfs[dictionary.token2id[w]]/158879))


Słowo 123 - polski pojawia się w korpusie 245334 razy i występuje w 73013 dokumentach (46.0%).


## Bag of Words
Dla każdego dokumentu tworzymy słownik zawierający informację jak dużo słów się pojawia i jak często.

In [22]:
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]

In [34]:
BOW_113 = BOW[3333]
for i in range(len(BOW_113)):
    if BOW_113[i][1]>1:
        v = "razy"
    else:
        v = "raz"
    print("Słowo {} ({}) pojawia się {} {}.".format(BOW_113[i][0], 
                                               dictionary[BOW_113[i][0]], 
BOW_113[i][1],v))

Słowo 10 (cywilny) pojawia się 2 razy.
Słowo 11 (czas) pojawia się 1 raz.
Słowo 14 (człowiek) pojawia się 1 raz.
Słowo 38 (istotny) pojawia się 1 raz.
Słowo 50 (kraj) pojawia się 1 raz.
Słowo 62 (musieć) pojawia się 3 razy.
Słowo 64 (myśleć) pojawia się 2 razy.
Słowo 83 (obcy) pojawia się 1 raz.
Słowo 89 (oczywiście) pojawia się 2 razy.
Słowo 99 (okres) pojawia się 1 raz.
Słowo 106 (państwo) pojawia się 4 razy.
Słowo 107 (pewien) pojawia się 4 razy.
Słowo 121 (polityczny) pojawia się 7 razy.
Słowo 122 (polska) pojawia się 5 razy.
Słowo 126 (poprzez) pojawia się 1 raz.
Słowo 131 (potrafić) pojawia się 2 razy.
Słowo 132 (powiedzieć) pojawia się 5 razy.
Słowo 142 (prawo) pojawia się 1 raz.
Słowo 156 (przykład) pojawia się 1 raz.
Słowo 169 (rzecz) pojawia się 2 razy.
Słowo 174 (sformułowanie) pojawia się 1 raz.
Słowo 175 (siebie) pojawia się 1 raz.
Słowo 180 (sprawa) pojawia się 1 raz.
Słowo 184 (stan) pojawia się 1 raz.
Słowo 194 (swój) pojawia się 4 razy.
Słowo 205 (trzy) pojawia się 1 r

## TF-IDF
Oblicznie wartości tf-idf

In [25]:
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.05216970571383937),
 (1, 0.0881694802798162),
 (2, 0.0761474678344046),
 (3, 0.0626284520099549),
 (4, 0.05464523351084474),
 (5, 0.04773424408337852),
 (6, 0.04964505554526941),
 (7, 0.04835387944372811),
 (8, 0.03276787170092533),
 (9, 0.0892553188157317),
 (10, 0.05820356617752606),
 (11, 0.0675207986060912),
 (12, 0.02405810760688753),
 (13, 0.11878449249954774),
 (14, 0.09888834718183552),
 (15, 0.03645244951711879),
 (16, 0.03320898634998834),
 (17, 0.05832580052930959),
 (18, 0.032797885692568),
 (19, 0.034367534567973874),
 (20, 0.007876942294595424),
 (21, 0.042937987456413494),
 (22, 0.01186162373357119),
 (23, 0.06799827083431338),
 (24, 0.028646366272486873),
 (25, 0.05874549154225238),
 (26, 0.048753491212853696),
 (27, 0.05133205606263977),
 (28, 0.03239943233493516),
 (29, 0.022996211808293834),
 (30, 0.019495019320528866),
 (31, 0.01767994108759474),
 (32, 0.028068602256199254),
 (33, 0.060480101276420783),
 (34, 0.052552706126542496),
 (35, 0.029732000857486996)

## LDA oparte o BOW

In [42]:
start = timer()
lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=100, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))


0:04:18.168380


In [43]:
funcs.print_topics(lda_model.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,...,Topic 90,Topic 91,Topic 92,Topic 93,Topic 94,Topic 95,Topic 96,Topic 97,Topic 98,Topic 99
0,karny,podatek,papier,organizacja,zmiana,polski,poseł,system,zdrowie,pieniądz,...,kolejowy,wojskowy,publiczny,prywatyzacja,knf,praca,roślina,energia,budżet,europejski
1,kara,podatkowy,górniczy,centrum,praca,język,narodowy,usługa,zdrowotny,polski,...,minister,obrona,prywatny,przedsiębiorstwo,nasienny,pracownik,odmiana,energetyczny,budżetowy,zmiana
2,przestępstwo,projekt,rynek,publiczny,projekt,cudzoziemiec,polski,elektroniczny,opieka,złoty,...,transport,żołnierz,prasowy,spółka,lwa,pracodawca,gmo,woda,wydatek,unia
3,kodeks,podatnik,finansowy,społeczny,przepis,obywatel,demokratyczny,telekomunikacyjny,medyczny,polska,...,bezpieczeństwo,narodowy,medium,górnictwo,chiński,inspekcja,stocznia,wodny,złoty,projekt
4,osoba,zmiana,wartościowy,działalność,proponować,polska,polityczny,dostęp,pacjent,mówić,...,policja,siła,prasa,skarb,szlachetny,państwowy,organizm,inwestycja,finanse,polski
5,wolność,stawka,geologiczny,pozarządowy,pracownik,kraj,naród,informacja,szpital,państwo,...,województwo,zbrojny,gier,państwo,lobbować,prawo,genetycznie,źródło,państwo,klub
6,przestępczość,osoba,obrót,pożytek,okres,pobyt,człowiek,operator,lekarz,rząd,...,pytanie,wojsko,dziennikarz,akcja,probierczy,umowa,samolot,elektryczny,publiczny,parlamentarny
7,sprawca,dochodowy,giełda,stowarzyszenie,dotyczyć,niedziela,lewica,poczta,lek,mld,...,przewóz,armia,działalność,restrukturyzacja,rotacja,zatrudnienie,materiał,gospodarka,dochód,pomoc
8,pozbawić,dochód,łowiecki,praca,rozwiązanie,terytorium,polska,internet,minister,minister,...,droga,służba,przekaz,państwowy,sekt,inspektor,siewny,odnawialny,wzrost,prawo
9,sprawiedliwość,skarbowy,koncesja,prawny,wprowadzić,handel,pamięć,informatyczny,ochrona,powiedzieć,...,kolej,polski,misja,kopalnia,market,zakład,rzemiosła,elektrownia,środek,sprawa


### Modyfikacje parametrów słownika

In [58]:
dictionary.filter_extremes(no_above=0.4, no_below=5 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]
start = timer()
lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=100, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))

0:04:10.242293


In [59]:
funcs.print_topics(lda_model.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,...,Topic 90,Topic 91,Topic 92,Topic 93,Topic 94,Topic 95,Topic 96,Topic 97,Topic 98,Topic 99
0,środek,morski,szkoła,zwierzę,prezydent,choroba,obywatelstwo,budżetowy,sieć,fundusz,...,wzrost,cena,pkp,rząd,wojskowy,usta,partia,bank,dokument,straż
1,budżet,wyrób,edukacja,produkt,narodowy,numer,polak,budżet,telekomunikacyjny,emerytalny,...,budżet,rynek,pociąg,polska,obrona,arta,polityczny,bankowy,materiał,graniczny
2,pieniądz,statek,szkół,zdrowie,zabytek,zwalczać,syndyk,płaca,operator,system,...,gospodarczy,produkcja,kolej,samolot,żołnierz,zapis,mandat,narodowy,informacja,pożarny
3,inwestycja,port,dziecko,alkohol,kultura,wirus,repatriacja,rząd,łódź,ubezpieczenie,...,gospodarka,paliw,wielkopolski,rosyjski,siła,zmiana,ordynacja,pieniężny,człowiek,wyznaniowy
4,fundusz,lotniczy,oświata,wyrób,marszałek,zakaźny,rodak,dochód,egzamin,składka,...,wydatek,paliwa,świętokrzyski,swój,narodowy,punkt,okręg,prezes,osoba,państwowy
5,powiedzieć,rybołówstwo,nauczyciel,badanie,kwaśniewski,świnia,granica,wzrost,telefon,program,...,poziom,cen,przejazd,premier,zbrojny,przepis,radny,polityka,nazwisko,związek
6,chodzić,morze,uczeń,środek,rzeczypospolity,identyfikacja,polska,wydatek,regulacyjny,środek,...,wysoki,polska,pkn,rosja,służba,proponować,wójt,nbp,wiedzieć,granica
7,autostrada,rybacki,młodzież,substancja,sejm,hodowca,kraj,sfer,komórkowy,inwestycyjny,...,dochód,import,poznań,obrona,wojsko,wniosek,wyborczy,centralny,powiedzieć,strażnik
8,rząd,rybak,szkolny,zakaz,polska,chorób,obywatel,wysoki,automata,reforma,...,publiczny,rząd,podwykonawca,polak,armia,artykuł,system,polska,prosić,ochotniczy
9,finansować,gospodarka,system,obrót,andrzej,afrykański,repatriant,społeczny,łódzki,społeczny,...,rząd,olej,orlen,bezpieczeństwo,rzeczypospolity,zakres,rozwiązanie,system,prawda,przejście


Liczba tematów - 20, odcięcie 50%

In [60]:
dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.5, no_below=5 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]
start = timer()
lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=20, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))

52425
0:02:03.008569


In [61]:
funcs.print_topics(lda_model.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19
0,prawo,bank,państwo,europejski,minister,środowisko,budżet,telewizja,las,podatek,projekt,poprawka,projekt,projekt,praca,polski,polska,sąd,poseł,zdrowie
1,dziecko,budżet,mówić,minister,państwo,szkoła,środek,rada,osoba,projekt,europejski,komisja,komisja,przepis,pracownik,narodowy,polski,sprawa,sejm,osoba
2,rodzina,rolny,rząd,polski,spółka,minister,mieszkaniowy,medium,polski,podatkowy,system,senat,poseł,prawo,projekt,polska,państwo,minister,komisja,minister
3,życie,złoty,powiedzieć,unia,prywatyzacja,energia,rząd,krajowy,sport,zmiana,przepis,klub,prawo,postępowanie,zmiana,naród,polityka,prawo,sprawa,dziecko
4,projekt,rolnictwo,poseł,polska,skarb,nauczyciel,gmina,publiczny,ochrona,gospodarczy,zmiana,zmiana,konstytucja,kontrola,zawodowy,wojskowy,kraj,służba,prawo,społeczny
5,konwencja,rolnik,minister,pytanie,pytanie,odpad,program,poseł,sportowy,działalność,prawo,samorząd,rada,zmiana,społeczny,rzeczypospolity,europejski,sprawiedliwość,marszałek,świadczenie
6,kobieta,kredyt,polski,rząd,przedsiębiorstwo,szkół,mieszkanie,kultura,leśny,osoba,dotyczyć,projekt,zmiana,organ,pracodawca,zostać,gospodarczy,policja,wniosek,praca
7,człowiek,środek,człowiek,rolnik,firma,energetyczny,inwestycja,radiofonia,polska,przedsiębiorca,ubezpieczenie,przyjąć,praca,sprawa,wynagrodzić,żołnierz,unia,rzecznik,izba,rodzina
8,państwo,polski,polska,kraj,mówić,ochrona,droga,swój,środowisko,rozwiązanie,zakres,stanowisko,wniosek,prawny,komisja,wojna,rząd,sędzia,wysoki,środek
9,kara,agencja,wiedzieć,rolnictwo,rząd,edukacja,złoty,polski,impreza,opłata,polski,dotyczyć,pytanie,dotyczyć,dotyczyć,kościół,sprawa,prokurator,klub,pomoc


In [62]:
start = timer()
dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.4, no_below=5 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=20, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))
a = funcs.print_topics(lda_model.print_topics(-1))

52413
0:03:03.868075


,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19
0,praca,budżet,gmina,świadczenie,dziecko,środowisko,sejm,sąd,poprawka,europejski,nauczyciel,bank,polska,spółka,podatek,służba,zdrowie,polska,zmiana,polska
1,pracownik,złoty,miasto,emerytalny,rodzina,ochrona,marszałek,konstytucyjny,senat,unia,szkoła,wniosek,dzień,prywatyzacja,podatkowy,urząd,zdrowotny,rząd,przepis,narodowy
2,państwowy,środek,województwo,osoba,życie,mieszkaniowy,powiedzieć,konstytucja,klub,polska,osoba,izba,swój,skarb,zmiana,informacja,lekarz,powiedzieć,samorząd,kraj
3,społeczny,rząd,samorząd,społeczny,człowiek,nieruchomość,rząd,trybunał,zmiana,kraj,edukacja,wysoki,kultura,przedsiębiorstwo,osoba,kontrola,medyczny,rolnik,arta,obrona
4,zawodowy,budżetowy,powiat,emerytura,kobieta,mieszkanie,prosić,postępowanie,przyjąć,rynek,nauka,bankowy,kościół,rada,przedsiębiorca,osoba,opieka,wiedzieć,praca,siła
5,pracodawca,finanse,mieszkaniec,system,rodzic,gmina,wniosek,wysoki,stanowisko,rolny,szkół,mniejszość,człowiek,telewizja,gospodarczy,ochrona,pacjent,pieniądz,usta,polityczny
6,inspekcja,wzrost,warszawa,rząd,rodzinny,las,polityczny,sprawiedliwość,parlamentarny,rozwój,niepełnosprawny,sprawozdanie,narodowy,program,dochód,organ,szpital,chodzić,proponować,naród
7,zatrudnienie,wydatek,rząd,ubezpieczenie,pomoc,własność,klub,prawny,nra,rząd,praca,kontrola,pamięć,krajowy,podatnik,administracja,lek,prosić,prawny,rząd
8,bezrobocie,fundusz,lokalny,emeryt,problem,grunt,głos,zasada,sejm,zakres,wysoki,finansowy,historia,rząd,rozwiązanie,dana,zakład,rolnictwo,zapis,polityka
9,zakład,kwota,region,drogowy,społeczny,gospodarka,wyborczy,sędzia,dzień,rolnictwo,system,sejm,jan,gospodarka,stawka,prezes,środek,musieć,zakres,polak


In [63]:
a = funcs.print_topics(lda_model.print_topics(-1))

In [67]:
a.T


,0,1,2,3,4,5,6,7,8,9
Topic 0,praca,pracownik,państwowy,społeczny,zawodowy,pracodawca,inspekcja,zatrudnienie,bezrobocie,zakład
Topic 1,budżet,złoty,środek,rząd,budżetowy,finanse,wzrost,wydatek,fundusz,kwota
Topic 2,gmina,miasto,województwo,samorząd,powiat,mieszkaniec,warszawa,rząd,lokalny,region
Topic 3,świadczenie,emerytalny,osoba,społeczny,emerytura,system,rząd,ubezpieczenie,emeryt,drogowy
Topic 4,dziecko,rodzina,życie,człowiek,kobieta,rodzic,rodzinny,pomoc,problem,społeczny
Topic 5,środowisko,ochrona,mieszkaniowy,nieruchomość,mieszkanie,gmina,las,własność,grunt,gospodarka
Topic 6,sejm,marszałek,powiedzieć,rząd,prosić,wniosek,polityczny,klub,głos,wyborczy
Topic 7,sąd,konstytucyjny,konstytucja,trybunał,postępowanie,wysoki,sprawiedliwość,prawny,zasada,sędzia
Topic 8,poprawka,senat,klub,zmiana,przyjąć,stanowisko,parlamentarny,nra,sejm,dzień
Topic 9,europejski,unia,polska,kraj,rynek,rolny,rozwój,rząd,zakres,rolnictwo


In [69]:
start = timer()
#dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.3, no_below=5 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=20, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))
a = funcs.print_topics(lda_model.print_topics(-1))

52397
0:02:44.803395


In [71]:
bow_30_20 = a

In [72]:
a

,0,1,2,3,4,5,6,7,8,9
Topic 0,sąd,sędzia,nauka,naukowy,uczelnia,sport,sprawiedliwość,sportowy,uniwersytet,sądownictwo
Topic 1,rolnik,rolnictwo,rolny,gospodarstwo,wieś,rolniczy,ziemia,agencja,hektar,dzień
Topic 2,bank,rynek,budżet,kredyt,finansowy,fundusz,publiczny,środek,gospodarka,gospodarczy
Topic 3,poprawka,senat,przyjąć,sejm,wniosek,stanowisko,nra,głosować,dzień,posiedzenie
Topic 4,podatek,podatkowy,dochód,finanse,gospodarczy,budżet,osoba,skarbowy,złoty,stawka
Topic 5,europejski,unia,kraj,polityka,europa,gospodarczy,musieć,droga,wielki,premier
Topic 6,konstytucja,samorząd,rada,konstytucyjny,wyborczy,gmina,prezydent,wybory,polityczny,sejm
Topic 7,karny,ochrona,kara,kodeks,przepis,przestępstwo,europejski,konwencja,środowisko,odpad
Topic 8,prosić,marszałek,wiedzieć,człowiek,sejm,premier,siebie,sala,musieć,naprawdę
Topic 9,służba,osoba,wojskowy,policja,żołnierz,obrona,straż,funkcjonariusz,bezpieczeństwo,narodowy


In [73]:
start = timer()
#dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.3, no_below=50 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=20, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))
funcs.print_topics(lda_model.print_topics(-1))

18200
0:02:07.546602


,0,1,2,3,4,5,6,7,8,9
Topic 0,europejski,unia,polityka,kraj,polityczny,telewizja,rada,zagraniczny,europa,musieć
Topic 1,sąd,konstytucyjny,trybunał,konstytucja,rzecznik,prawny,sprawiedliwość,sędzia,postępowanie,zasada
Topic 2,rolnik,rolnictwo,rolny,środek,program,agencja,gospodarstwo,wieś,rolniczy,produkcja
Topic 3,droga,drogowy,transport,inwestycja,infrastruktura,ruch,pojazd,autostrada,kolejowy,środek
Topic 4,prosić,wiedzieć,człowiek,premier,marszałek,siebie,naprawdę,musieć,zrobić,pieniądz
Topic 5,poprawka,senat,przyjąć,stanowisko,parlamentarny,głosować,nra,odrzucić,imię,przedstawić
Topic 6,spółka,mieszkaniowy,skarb,nieruchomość,własność,mieszkanie,energia,przedsiębiorstwo,gospodarka,majątek
Topic 7,sejm,wniosek,izba,marszałek,posiedzenie,poprawka,przyjąć,sprawozdanie,dzień,czytać
Topic 8,kultura,nauka,naukowy,rozwój,uczelnia,miasto,narodowy,kraj,uniwersytet,wielki
Topic 9,samorząd,gmina,rada,terytorialny,usta,administracja,zadanie,organ,jednostka,samorządowy


In [74]:
bow_30_50_20 = funcs.print_topics(lda_model.print_topics(-1))

In [75]:
tematy = ['polityka UE','sądownictwo','rolnictwo','transport','topic 4', 'sejm i senat', 'mieszkaniowe', 'sejm i senat', 'kultura i nauka',
         'administracja','środowisko', 'prawo', 'polityka UE', 'służba zdrowia', 'narodowy', 'oświata','rodzina','budżet','podatki','finanse']

In [78]:
bow_30_50_20.index = tematy
bow_30_50_20

,0,1,2,3,4,5,6,7,8,9
polityka UE,europejski,unia,polityka,kraj,polityczny,telewizja,rada,zagraniczny,europa,musieć
sądownictwo,sąd,konstytucyjny,trybunał,konstytucja,rzecznik,prawny,sprawiedliwość,sędzia,postępowanie,zasada
rolnictwo,rolnik,rolnictwo,rolny,środek,program,agencja,gospodarstwo,wieś,rolniczy,produkcja
transport,droga,drogowy,transport,inwestycja,infrastruktura,ruch,pojazd,autostrada,kolejowy,środek
topic 4,prosić,wiedzieć,człowiek,premier,marszałek,siebie,naprawdę,musieć,zrobić,pieniądz
sejm i senat,poprawka,senat,przyjąć,stanowisko,parlamentarny,głosować,nra,odrzucić,imię,przedstawić
mieszkaniowe,spółka,mieszkaniowy,skarb,nieruchomość,własność,mieszkanie,energia,przedsiębiorstwo,gospodarka,majątek
sejm i senat,sejm,wniosek,izba,marszałek,posiedzenie,poprawka,przyjąć,sprawozdanie,dzień,czytać
kultura i nauka,kultura,nauka,naukowy,rozwój,uczelnia,miasto,narodowy,kraj,uniwersytet,wielki
administracja,samorząd,gmina,rada,terytorialny,usta,administracja,zadanie,organ,jednostka,samorządowy


In [79]:
start = timer()
dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.3, no_below=50 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=35, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))
funcs.print_topics(lda_model.print_topics(-1))

18200
0:03:24.034137


,0,1,2,3,4,5,6,7,8,9
Topic 0,środowisko,ochrona,odpad,woda,wodny,gospodarka,ekologiczny,naturalny,zasób,opłata
Topic 1,pracownik,pracodawca,zawodowy,zakład,państwowy,umowa,pracowniczy,wynagrodzić,społeczny,zatrudnienie
Topic 2,służba,policja,straż,funkcjonariusz,celny,wewnętrzny,bezpieczeństwo,ochrona,administracja,graniczny
Topic 3,usta,arta,przepis,zapis,proponować,punkt,zakres,prawny,zasada,dzień
Topic 4,podatkowy,podatek,przepis,przedsiębiorca,publiczny,działalność,gospodarczy,skarbowy,podmiot,podatnik
Topic 5,dziecko,rodzina,kobieta,pomoc,rodzinny,społeczny,rodzic,problem,osoba,życie
Topic 6,rolnik,rolnictwo,rolny,gospodarstwo,produkcja,rolniczy,rynek,wieś,agencja,producent
Topic 7,ruch,drogowy,pojazd,samochód,kierowca,droga,wypadek,bezpieczeństwo,transport,prezydent
Topic 8,droga,inwestycja,krajowy,autostrada,budowy,infrastruktura,budowa,realizacja,program,środek
Topic 9,karny,kara,kodeks,przestępstwo,przepis,postępowanie,wolność,osoba,sprawca,pozbawić


In [80]:
bow_30_50_50 = funcs.print_topics(lda_model.print_topics(-1))

## LDA oparte o TF-IDF

In [66]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"
Topic: 1 Word: 0.005*"rolnik" + 0.004*"rolnictwo" + 0.004*"rolny" + 0.004*"europejski" + 0.003*"unia" + 0.003*"poprawka" + 0.002*"rynek" + 0.002*"rozwój" + 0.002*"podatek" + 0.002*"gospodarstwo"
Topic: 2 Word: 0.002*"rząd" + 0.002*"polska" + 0.002*"premier" + 0.002*"marszałek" + 0.002*"człowiek" + 0.002*"sejm" + 0.002*"prosić" + 0.002*"polityczny" + 0.001*"powiedzieć" + 0.001*"polak"
Topic: 3 Word: 0.003*"dziecko" + 0.003*"praca" + 0.003*"osoba" + 0.002*"zdrowie" + 0.002*"zmiana" + 0.002*"poprawka" + 0.002*"służba" + 0.002*"sąd" + 0.002*"świadczenie" + 0.002*"rodzina"
Topic: 4 Word: 0.011*"poprawka" + 0.006*"senat" + 0.003*"klub" + 0.003*"zmiana" + 0.003*"przyjąć" + 0.002*"głosować" + 0.002*"sprawozdanie" + 0.002*"nieruchomość" + 0.002*"zgłosić" + 0.002*"konwencja"


In [69]:
lda_model_tfidf.print_topics(0)

[(0,
  '0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"'),
 (3,
  '0.003*"dziecko" + 0.003*"praca" + 0.003*"osoba" + 0.002*"zdrowie" + 0.002*"zmiana" + 0.002*"poprawka" + 0.002*"służba" + 0.002*"sąd" + 0.002*"świadczenie" + 0.002*"rodzina"'),
 (4,
  '0.011*"poprawka" + 0.006*"senat" + 0.003*"klub" + 0.003*"zmiana" + 0.003*"przyjąć" + 0.002*"głosować" + 0.002*"sprawozdanie" + 0.002*"nieruchomość" + 0.002*"zgłosić" + 0.002*"konwencja"'),
 (2,
  '0.002*"rząd" + 0.002*"polska" + 0.002*"premier" + 0.002*"marszałek" + 0.002*"człowiek" + 0.002*"sejm" + 0.002*"prosić" + 0.002*"polityczny" + 0.001*"powiedzieć" + 0.001*"polak"'),
 (1,
  '0.005*"rolnik" + 0.004*"rolnictwo" + 0.004*"rolny" + 0.004*"europejski" + 0.003*"unia" + 0.003*"poprawka" + 0.002*"rynek" + 0.002*"rozwój" + 0.002*"podatek" + 0.002*"gospodarstwo"')]

In [100]:
funcs.print_topics(lda_model_tfidf.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja


In [85]:
lda_model_tfidf.print_topics(-1)[0]

(0,
 '0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"')

In [89]:
pd.Series(a[0][1].split(' + ')).apply(lambda x : re.sub(r'[\d\.\*\"\']', '', x))

0        poprawka
1           senat
2            klub
3          zmiana
4         przyjąć
5        głosować
6    sprawozdanie
7    nieruchomość
8         zgłosić
9       konwencja
Name: Topic 4, dtype: object

In [91]:
topic_df = pd.DataFrame()
for topic in lda_model_tfidf.print_topics(-1):
    topic_df['Topic ' + str(topic[0])] = pd.Series(topic[1].split(' + ')).apply(lambda x : re.sub(r'[\d\.\*\"\']', '', x))

In [93]:
topic_df

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja
